#CARGA DE DATOS

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd

ruta_archivo = "/content/drive/MyDrive/Investigación/Data Entrenamiento.xlsx"
DataEnt = pd.read_excel(ruta_archivo)
DataEnt.head()

,tweet,Date,sentimiento,sentimiento1
0,Antes de Petro ya existía Colombia. Por eso es...,NaN,negativo,-1.0
1,OIT no apoya contenido de la reforma laboral d...,NaN,negativo,-0.9
2,El Presidente recibía una economía que apenas ...,NaN,positivo,0.9
3,Reformas pensional y laboral del Gobierno Petr...,NaN,NaN,0.6
4,Mientras el presidente Gustavo Petro le dice a...,NaN,neutro,0.5


In [4]:
!pip install transformers
!pip install matplotlib
!pip install textblob

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 37.2 MB/s eta 0:00:00


#Léxico


##VADER
(Valence Aware Dictionary and sEntiment Reasoner)
##SentiWordNet
(Base de datos léxica para asignar valores de polaridad de sentimiento)

In [5]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('sentiwordnet')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package sentiwordnet to /root/nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!


True

In [6]:
!pip install vaderSentiment

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 2.9 MB/s eta 0:00:00


In [7]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [9]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize
from nltk.corpus import sentiwordnet as swn
import nltk
import torch
import numpy as np

# Descargar el recurso Spanish stopwords
nltk.download('stopwords')

# Crear una instancia del analizador de sentimientos de Vader
analyzer = SentimentIntensityAnalyzer()

# Cargar el tokenizador y el modelo preentrenado de BERT en español
tokenizer = BertTokenizer.from_pretrained('dccuchile/bert-base-spanish-wwm-cased')
model = BertForSequenceClassification.from_pretrained('dccuchile/bert-base-spanish-wwm-cased', num_labels=1)

# Función para analizar el sentimiento de un texto usando Vader
def analyze_vader_sentiment(text):
    sentiment = analyzer.polarity_scores(text)
    return sentiment['compound']

# Función para analizar el sentimiento de un texto usando SentiWordNet
def analyze_sentiwordnet_sentiment(text):
    tokens = word_tokenize(text.lower(), language='spanish')
    sentiment_score = 0

    for token in tokens:
        synsets = list(swn.senti_synsets(token))
        if synsets:
            sentiment = synsets[0]
            sentiment_score += sentiment.pos_score() - sentiment.neg_score()

    return sentiment_score

# Ruta del archivo Excel
ruta_archivo = "/content/drive/MyDrive/Investigación/Data Entrenamiento.xlsx"

# Leer el archivo Excel y seleccionar la columna "tweet"
data = pd.read_excel(ruta_archivo)
column_name = 'tweet'

# Inicializar listas para almacenar las puntuaciones
vader_scores = []
sentiwordnet_scores = []
bert_scores = []

for index, row in data.iterrows():
    text = row[column_name]
    vader_score = analyze_vader_sentiment(text)
    sentiwordnet_score = analyze_sentiwordnet_sentiment(text)

    # Tokenizar y convertir el texto en tensor para BERT
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']

    # Realizar la predicción con BERT y asignar la puntuación de sentimiento
    with torch.no_grad():
        output = model(input_ids, attention_mask=attention_mask)[0]

    # Agregar la puntuación de BERT a la lista
    bert_score = output.item()
    bert_scores.append(bert_score)

    vader_scores.append(vader_score)
    sentiwordnet_scores.append(sentiwordnet_score)

# Escalar las puntuaciones de BERT al rango de -1 a 1
min_bert_score = min(bert_scores)
max_bert_score = max(bert_scores)
bert_scores_normalized = [2 * (score - min_bert_score) / (max_bert_score - min_bert_score) - 1 for score in bert_scores]

# Asignar las puntuaciones a las listas correspondientes
data['vader_sentiment_score'] = vader_scores
data['sentiwordnet_sentiment_score'] = sentiwordnet_scores
data['bert_sentiment_score'] = bert_scores_normalized

# Guardar los resultados en un nuevo archivo Excel
result_file = "/content/drive/MyDrive/Investigación/Resultados_Sentimiento.xlsx"
data.to_excel(result_file, index=False)

print("Resultados guardados en:", result_file)



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Resultados guardados en: /content/drive/MyDrive/Investigación/Resultados_Sentimiento.xlsx


##EmoLex

In [ ]:
import pandas as pd
from nltk.tokenize import word_tokenize
import nltk

# Descargar el recurso Spanish stopwords
nltk.download('stopwords')

# Función para cargar EmoLex
def load_emolex(emolex_file):
    emolex = {}
    with open(emolex_file, 'r') as file:
        for line in file:
            word, emotion, value = line.strip().split()
            if emotion == 'joy':
                emolex[word] = float(value)
    return emolex

# Función para analizar el sentimiento de un texto usando EmoLex
def analyze_emolex_sentiment(text, emolex_lexicon):
    tokens = word_tokenize(text.lower(), language='spanish')
    sentiment_score = 0

    for token in tokens:
        if token in emolex_lexicon:
            sentiment_score += emolex_lexicon[token]

    return sentiment_score

# Ruta del archivo Excel
ruta_archivo = "/content/drive/MyDrive/Investigación/Data Entrenamiento.xlsx"

# Leer el archivo Excel y seleccionar la columna "tweet"
data = pd.read_excel(ruta_archivo)
column_name = 'tweet'

# Cargar EmoLex
emolex_file = "ruta_al_archivo_emolex.txt"  # Reemplaza con la ruta al archivo descargado
emolex_lexicon = load_emolex(emolex_file)

# Inicializar lista para almacenar las puntuaciones
emolex_scores = []

for index, row in data.iterrows():
    text = row[column_name]
    emolex_score = analyze_emolex_sentiment(text, emolex_lexicon)
    emolex_scores.append(emolex_score)

# Asignar las puntuaciones a la lista correspondiente
data['emolex_sentiment_score'] = emolex_scores

# Guardar los resultados en un nuevo archivo Excel
result_file = "/content/drive/MyDrive/Investigación/Resultados_Emolex_Sentimiento.xlsx"
data.to_excel(result_file, index=False)

print("Resultados de EmoLex guardados en:", result_file)


##Opinion Lexicon

In [ ]:
import pandas as pd
from nltk.corpus import opinion_lexicon
from nltk.tokenize import word_tokenize
import nltk

# Descargar el recurso Spanish stopwords
nltk.download('stopwords')

# Función para cargar Opinion Lexicon de NLTK
pos_words = set(opinion_lexicon.positive())
neg_words = set(opinion_lexicon.negative())

# Función para analizar el sentimiento de un texto usando Opinion Lexicon
def analyze_opinion_lexicon_sentiment(text):
    tokens = word_tokenize(text.lower(), language='spanish')
    sentiment_score = 0

    for token in tokens:
        if token in pos_words:
            sentiment_score += 1
        elif token in neg_words:
            sentiment_score -= 1

    return sentiment_score

# Ruta del archivo Excel
ruta_archivo = "/content/drive/MyDrive/Investigación/Data Entrenamiento.xlsx"

# Leer el archivo Excel y seleccionar la columna "tweet"
data = pd.read_excel(ruta_archivo)
column_name = 'tweet'

# Inicializar lista para almacenar las puntuaciones
opinion_lexicon_scores = []

for index, row in data.iterrows():
    text = row[column_name]
    opinion_lexicon_score = analyze_opinion_lexicon_sentiment(text)
    opinion_lexicon_scores.append(opinion_lexicon_score)

# Asignar las puntuaciones a la lista correspondiente
data['opinion_lexicon_sentiment_score'] = opinion_lexicon_scores

# Guardar los resultados en un nuevo archivo Excel
result_file = "/content/drive/MyDrive/Investigación/Resultados_Opinion_Lexicon_Sentimiento.xlsx"
data.to_excel(result_file, index=False)

print("Resultados del Opinion Lexicon guardados en:", result_file)


##NRC Emotion Lexicon

In [ ]:
import pandas as pd
from nltk.tokenize import word_tokenize
import nltk

# Descargar el recurso Spanish stopwords
nltk.download('stopwords')

# Función para cargar el NRC Emotion Lexicon
def load_nrc_emotion_lexicon(nrc_emotion_file):
    nrc_emotion_lexicon = {}
    with open(nrc_emotion_file, 'r') as file:
        for line in file:
            word, emotion, value = line.strip().split()
            if emotion not in nrc_emotion_lexicon:
                nrc_emotion_lexicon[emotion] = {}
            nrc_emotion_lexicon[emotion][word] = int(value)
    return nrc_emotion_lexicon

# Función para analizar el sentimiento de un texto usando NRC Emotion Lexicon
def analyze_nrc_emotion_sentiment(text, nrc_emotion_lexicon):
    tokens = word_tokenize(text.lower(), language='spanish')
    emotion_scores = {emotion: 0 for emotion in nrc_emotion_lexicon.keys()}

    for token in tokens:
        for emotion in nrc_emotion_lexicon.keys():
            if token in nrc_emotion_lexicon[emotion]:
                emotion_scores[emotion] += nrc_emotion_lexicon[emotion][token]

    return emotion_scores

# Ruta del archivo Excel
ruta_archivo = "/content/drive/MyDrive/Investigación/Data Entrenamiento.xlsx"

# Leer el archivo Excel y seleccionar la columna "tweet"
data = pd.read_excel(ruta_archivo)
column_name = 'tweet'

# Cargar NRC Emotion Lexicon
nrc_emotion_file = "ruta_al_archivo_nrc_emotion_lexicon.txt"  # Reemplaza con la ruta al archivo NRC Emotion Lexicon descargado
nrc_emotion_lexicon = load_nrc_emotion_lexicon(nrc_emotion_file)

# Inicializar lista para almacenar las puntuaciones de emociones
emotion_scores_list = []

for index, row in data.iterrows():
    text = row[column_name]
    emotion_scores = analyze_nrc_emotion_sentiment(text, nrc_emotion_lexicon)
    emotion_scores_list.append(emotion_scores)

# Crear un DataFrame de las puntuaciones de emociones
emotion_df = pd.DataFrame(emotion_scores_list)

# Asignar las puntuaciones a las listas correspondientes
for emotion in emotion_df.columns:
    data[f'nrc_emotion_{emotion}_score'] = emotion_df[emotion]

# Guardar los resultados en un nuevo archivo Excel
result_file = "/content/drive/MyDrive/Investigación/Resultados_NRC_Emotion_Sentimiento.xlsx"
data.to_excel(result_file, index=False)

print("Resultados del NRC Emotion Lexicon guardados en:", result_file)


#BERT
Bidirectional Encoder Representations from Transformers


In [ ]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Cargar el conjunto de datos desde un archivo Excel
ruta_archivo = "/content/drive/MyDrive/Investigación/Data Entrenamiento.xlsx"
data = pd.read_excel(ruta_archivo)

# Ajustar los valores de sentimiento al rango de -1 a 1
data['sentimiento1'] = ((data['sentimiento1'] - 2) / 2).astype(float)

# Cargar el tokenizador y el modelo preentrenado de BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=1)

# Poner el modelo en modo de evaluación
model.eval()

# Iterar a través de cada tweet y realizar predicciones
bert_sentiment_scores = []  # Lista para almacenar las predicciones de BERT

for i, row in data.iterrows():
    tweet = row['tweet']
    # Tokenizar y convertir el tweet en tensor
    inputs = tokenizer(tweet, return_tensors="pt", padding=True, truncation=True)
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']

    # Realizar la predicción
    with torch.no_grad():
        output = model(input_ids, attention_mask=attention_mask)[0]

    # Escalar la predicción al rango de -1 a 1
    prediction = output.item()
    scaled_prediction = 2 * (prediction - min_prediction) / (max_prediction - min_prediction) - 1

    bert_sentiment_scores.append(scaled_prediction)  # Agregar la predicción escalada a la lista

# Crear una nueva columna en el DataFrame para las puntuaciones de BERT
data['bert_sentiment_score_s'] = bert_sentiment_scores

# Guardar los resultados en un nuevo archivo Excel
result_file = "/content/drive/MyDrive/Investigación/Resultados_Sentimiento.xlsx"
data.to_excel(result_file, index=False)

print("Resultados guardados en:", result_file)



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Resultados guardados en: /content/drive/MyDrive/Investigación/Resultados_Sentimiento.xlsx


#Transformadores

In [ ]:
!pip install transformers
!pip install matplotlib
!pip install textblob
!pip install xformers

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
import torch

# Cargar el conjunto de datos desde un archivo Excel
ruta_archivo = "/content/drive/MyDrive/Investigación/Data Entrenamiento.xlsx"
data = pd.read_excel(ruta_archivo)

# Cargar el tokenizador y el modelo preentrenado de BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=1)  # Usar num_labels=1 para una sola unidad de clasificación

# Definir DataLoader para entrenamiento y prueba
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

# Definir función de pérdida y optimizador
criterion = torch.nn.MSELoss()  # Usar MSE (Mean Squared Error) para la regresión
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

# Resto del código (entrenar y evaluar el modelo)...

# Poner el modelo en modo de evaluación
model.eval()

# Iterar a través de cada tweet y realizar predicciones
bert_sentiment_scores = []  # Lista para almacenar las predicciones de BERT

for i, row in data.iterrows():
    tweet = row['tweet']
    # Tokenizar y convertir el tweet en tensor
    inputs = tokenizer(tweet, return_tensors="pt", padding=True, truncation=True)
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']

    # Realizar la predicción
    with torch.no_grad():
        output = model(input_ids, attention_mask=attention_mask).logits.squeeze().item()  # Usar .logits y .squeeze() para obtener un valor escalar

    bert_sentiment_scores.append(output)  # Agregar la predicción a la lista

# Asegurarnos de que las puntuaciones tengan la misma longitud que el conjunto de datos
assert len(bert_sentiment_scores) == len(data)

# Agregar las puntuaciones de sentimiento de BERT a una nueva columna
data['bert_sentiment_score'] = bert_sentiment_scores

# Guardar los resultados en un nuevo archivo Excel
result_file = "/content/drive/MyDrive/Investigación/Resultados_Sentimiento.xlsx"
data.to_excel(result_file, index=False)

print("Resultados guardados en:", result_file)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Resultados guardados en: /content/drive/MyDrive/Investigación/Resultados_Sentimiento.xlsx


## RoBERTa
A Robustly Optimized BERT Pretraining Approach



In [ ]:
# Instala las bibliotecas si aún no las tienes instaladas
# pip install transformers
# pip install torch

import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification

# Carga el modelo preentrenado y el tokenizador
model_name = "roberta-base"  # Puedes cambiar el modelo según tus necesidades
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(model_name)

# Define el texto que deseas analizar
texto = "Este es un ejemplo de análisis de sentimiento."

# Tokeniza el texto y lo convierte en un tensor
inputs = tokenizer(texto, return_tensors="pt", padding=True, truncation=True)

# Realiza la clasificación de sentimiento
with torch.no_grad():
    outputs = model(**inputs)

# Obtiene las probabilidades de clasificación
logits = outputs.logits

# Convierte las probabilidades en una etiqueta de sentimiento
predicted_class = torch.argmax(logits, dim=1).item()

# Define una lista de etiquetas de sentimiento
etiquetas_sentimiento = ["Negativo", "Neutral", "Positivo"]

# Muestra el resultado
print(f"Sentimiento: {etiquetas_sentimiento[predicted_class]}")


##DistilBERT


In [ ]:
# Instala las bibliotecas si aún no las tienes instaladas
# pip install transformers
# pip install torch

import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

# Carga el modelo preentrenado y el tokenizador
model_name = "distilbert-base-uncased"  # Puedes cambiar el modelo según tus necesidades
tokenizer = DistilBertTokenizer.from_pretrained(model_name)
model = DistilBertForSequenceClassification.from_pretrained(model_name)

# Define el texto que deseas analizar
texto = "Este es un ejemplo de análisis de sentimiento."

# Tokeniza el texto y lo convierte en un tensor
inputs = tokenizer(texto, return_tensors="pt", padding=True, truncation=True)

# Realiza la clasificación de sentimiento
with torch.no_grad():
    outputs = model(**inputs)

# Obtiene las probabilidades de clasificación
logits = outputs.logits

# Convierte las probabilidades en una etiqueta de sentimiento
predicted_class = torch.argmax(logits, dim=1).item()

# Define una lista de etiquetas de sentimiento
etiquetas_sentimiento = ["Negativo", "Neutral", "Positivo"]

# Muestra el resultado
print(f"Sentimiento: {etiquetas_sentimiento[predicted_class]}")


##GPT

(Generative Pre-trained Transformer)

In [ ]:
# Instala las bibliotecas si aún no las tienes instaladas
# pip install transformers
# pip install torch

import torch
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification

# Carga el tokenizador y el modelo preentrenado GPT-2
model_name = "gpt2"  # Puedes cambiar el modelo según tus necesidades
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2ForSequenceClassification.from_pretrained(model_name)

# Define el texto que deseas analizar
texto = "Este es un ejemplo de análisis de sentimiento."

# Tokeniza el texto y lo convierte en un tensor
inputs = tokenizer(texto, return_tensors="pt", padding=True, truncation=True)

# Realiza la clasificación de sentimiento
with torch.no_grad():
    outputs = model(**inputs)

# Obtiene las probabilidades de clasificación
logits = outputs.logits

# Convierte las probabilidades en una etiqueta de sentimiento
predicted_class = torch.argmax(logits, dim=1).item()

# Define una lista de etiquetas de sentimiento
etiquetas_sentimiento = ["Negativo", "Neutral", "Positivo"]

# Muestra el resultado
print(f"Sentimiento: {etiquetas_sentimiento[predicted_class]}")


##XLNet


In [ ]:
# Instala las bibliotecas si aún no las tienes instaladas
# pip install transformers
# pip install torch

import torch
from transformers import XLNetTokenizer, XLNetForSequenceClassification

# Carga el modelo preentrenado y el tokenizador
model_name = "xlnet-base-cased"  # Puedes cambiar el modelo según tus necesidades
tokenizer = XLNetTokenizer.from_pretrained(model_name)
model = XLNetForSequenceClassification.from_pretrained(model_name)

# Define el texto que deseas analizar
texto = "Este es un ejemplo de análisis de sentimiento."

# Tokeniza el texto y lo convierte en un tensor
inputs = tokenizer(texto, return_tensors="pt", padding=True, truncation=True)

# Realiza la clasificación de sentimiento
with torch.no_grad():
    outputs = model(**inputs)

# Obtiene las probabilidades de clasificación
logits = outputs.logits

# Convierte las probabilidades en una etiqueta de sentimiento
predicted_class = torch.argmax(logits, dim=1).item()

# Define una lista de etiquetas de sentimiento
etiquetas_sentimiento = ["Negativo", "Neutral", "Positivo"]

# Muestra el resultado
print(f"Sentimiento: {etiquetas_sentimiento[predicted_class]}")


##T5
(Text-To-Text Transfer Transformer)

In [ ]:
# Instala las bibliotecas si aún no las tienes instaladas
# pip install transformers
# pip install torch

import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Carga el modelo preentrenado y el tokenizador
model_name = "t5-small"  # Puedes cambiar el modelo según tus necesidades
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Define el texto que deseas analizar
texto = "Este es un ejemplo de análisis de sentimiento."

# Formato de entrada y salida para T5
input_text = "sentiment: " + texto + " </s>"

# Tokeniza el texto y lo convierte en un tensor
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

# Realiza la generación de texto para obtener la etiqueta de sentimiento
with torch.no_grad():
    output = model.generate(input_ids, max_length=2)

# Convierte la salida en texto
predicted_label = tokenizer.decode(output[0], skip_special_tokens=True)

# Define una lista de etiquetas de sentimiento
etiquetas_sentimiento = ["Negativo", "Neutral", "Positivo"]

# Muestra el resultado
print(f"Sentimiento: {etiquetas_sentimiento[int(predicted_label)]}")


##ELECTRA
(Efficiently Learning an Encoder that Classifies Token Replacements Accurately)

In [ ]:
# Instala las bibliotecas si aún no las tienes instaladas
# pip install transformers
# pip install torch

import torch
from transformers import ElectraTokenizer, ElectraForSequenceClassification

# Carga el modelo preentrenado y el tokenizador
model_name = "google/electra-base-discriminator"  # Puedes cambiar el modelo según tus necesidades
tokenizer = ElectraTokenizer.from_pretrained(model_name)
model = ElectraForSequenceClassification.from_pretrained(model_name)

# Define el texto que deseas analizar
texto = "Este es un ejemplo de análisis de sentimiento."

# Tokeniza el texto y lo convierte en un tensor
inputs = tokenizer(texto, return_tensors="pt", padding=True, truncation=True)

# Realiza la clasificación de sentimiento
with torch.no_grad():
    outputs = model(**inputs)

# Obtiene las probabilidades de clasificación
logits = outputs.logits

# Convierte las probabilidades en una etiqueta de sentimiento
predicted_class = torch.argmax(logits, dim=1).item()

# Define una lista de etiquetas de sentimiento
etiquetas_sentimiento = ["Negativo", "Neutral", "Positivo"]

# Muestra el resultado
print(f"Sentimiento: {etiquetas_sentimiento[predicted_class]}")


##BART (BART: Denoising Sequence-to-Sequence Pretraining for Natural Language Generation, Translation, and Comprehension)

In [ ]:
# Instala las bibliotecas si aún no las tienes instaladas
# pip install transformers
# pip install torch

import torch
from transformers import BartTokenizer, BartForConditionalGeneration

# Carga el modelo preentrenado y el tokenizador
model_name = "facebook/bart-large-cnn"  # Puedes cambiar el modelo según tus necesidades
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

# Define el texto que deseas analizar
texto = "Este es un ejemplo de análisis de sentimiento."

# Formato de entrada y salida para BART
input_text = "sentiment: " + texto + "</s>"

# Tokeniza el texto y lo convierte en un tensor
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

# Realiza la generación de texto para obtener la etiqueta de sentimiento
with torch.no_grad():
    output = model.generate(input_ids, max_length=2)

# Convierte la salida en texto
predicted_label = tokenizer.decode(output[0], skip_special_tokens=True)

# Define una lista de etiquetas de sentimiento
etiquetas_sentimiento = ["Negativo", "Neutral", "Positivo"]

# Muestra el resultado
print(f"Sentimiento: {etiquetas_sentimiento[int(predicted_label)]}")


##CTRL (Conditional Transformer Language Model)

In [ ]:
# Instala las bibliotecas si aún no las tienes instaladas
# pip install transformers
# pip install torch

import torch
from transformers import CTRLTokenizer, CTRLModel

# Carga el modelo preentrenado y el tokenizador
model_name = "SalesforceCTRL/ctrl"  # Puedes cambiar el modelo según tus necesidades
tokenizer = CTRLTokenizer.from_pretrained(model_name)
model = CTRLModel.from_pretrained(model_name)

# Define el texto que deseas analizar
texto = "Este es un ejemplo de análisis de sentimiento."

# Tokeniza el texto y lo convierte en un tensor
inputs = tokenizer(texto, return_tensors="pt", padding=True, truncation=True)

# Realiza la clasificación de sentimiento
with torch.no_grad():
    outputs = model(**inputs)

# Puedes procesar los outputs como desees, ya que CTRL no está diseñado para tareas específicas de clasificación de sentimiento.

# Muestra el resultado
print(outputs)


##GloVe (Global Vectors for Word Representation)

In [ ]:
# Carga los vectores GloVe preentrenados
word_vectors = {}
glove_file = 'glove.6B.50d.txt'  # Ruta al archivo GloVe que descargaste
with open(glove_file, 'r', encoding='utf-8') as file:
    for line in file:
        values = line.split()
        word = values[0]
        vector = [float(val) for val in values[1:]]
        word_vectors[word] = vector

# Define una función para calcular el sentimiento promedio de un texto
def calculate_sentiment(text):
    words = text.split()
    sentiment_vector = [0.0] * len(word_vectors['the'])  # Inicializa un vector de sentimiento

    for word in words:
        if word in word_vectors:
            word_vector = word_vectors[word]
            sentiment_vector = [a + b for a, b in zip(sentiment_vector, word_vector)]

    if len(words) > 0:
        sentiment_vector = [x / len(words) for x in sentiment_vector]

    return sentiment_vector

# Ejemplo de uso
texto = "La economía está creciendo y las inversiones están aumentando."
sentiment_vector = calculate_sentiment(texto)

# Define una polaridad de sentimiento basada en el vector resultante
def determine_sentiment(polarity_vector):
    if polarity_vector[0] > 0.2:
        return "Positivo"
    elif polarity_vector[0] < -0.2:
        return "Negativo"
    else:
        return "Neutral"

sentimiento = determine_sentiment(sentiment_vector)

# Muestra el resultado
print(f"Texto: '{texto}'")
print(f"Sentimiento: {sentimiento}")


#Redes Neuronales

## Red neuronal recurrente (RNN)

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Datos de ejemplo: textos y etiquetas de sentimiento
textos = ["Este es un buen producto.",
          "No me gusta esta película en absoluto.",
          "La comida en este restaurante es deliciosa.",
          "El servicio al cliente fue terrible.",
          "Me siento muy feliz hoy."]

etiquetas = [1, 0, 1, 0, 1]  # 1 para sentimiento positivo, 0 para sentimiento negativo

# Tokenización y secuencias
tokenizer = Tokenizer(num_words=1000, oov_token='<OOV>')  # Limita el vocabulario a 1000 palabras
tokenizer.fit_on_texts(textos)
secuencias = tokenizer.texts_to_sequences(textos)
secuencias_padded = pad_sequences(secuencias, maxlen=10, padding='post', truncating='post')

# Creación del modelo
model = Sequential()
model.add(Embedding(input_dim=1000, output_dim=16, input_length=10))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrenamiento del modelo
model.fit(secuencias_padded, etiquetas, epochs=10, verbose=2)

# Ejemplo de clasificación de sentimiento
nuevo_texto = ["Me encanta este libro."]
secuencia_nuevo_texto = tokenizer.texts_to_sequences(nuevo_texto)
secuencia_nuevo_texto_padded = pad_sequences(secuencia_nuevo_texto, maxlen=10, padding='post', truncating='post')

resultado = model.predict(secuencia_nuevo_texto_padded)

if resultado[0] >= 0.5:
    print("Sentimiento positivo")
else:
    print("Sentimiento negativo")
